In [1]:
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Ge

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install numba

from numba import cuda
device = cuda.get_current_device()
print(device)

<CUDA device 0 'b'Tesla T4''>


## Generate LLama2 Model





In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Create The Pipeline from the loaded Llama2 Model and tokenizer

In [5]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.01,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# Ask something

In [7]:
print(local_llm('What do you know about Llama2? Answer short in one sentence.'))



Llama2 is a cryptocurrency that was created in 2019 as an alternative to the popular privacy-focused coin, Monero (XMR). While both coins share some similarities, there are several key differences between them. Here are some of the main differences:
1. Mining Algorithm: Llama2 uses the CryptoNightV7 mining algorithm, which is different from Monero's Random Xorshift++ algorithm. This means that Llama2 has a different proof-of-work mechanism and may be more or less difficult to mine depending on the hardware used.
2. Block Time: The block time for Llama2 is shorter than that of Monero, at around 60 seconds compared to Monero's 2 minutes. This faster block time allows for more frequent transactions and potentially higher throughput.
3. Transaction Capacity: Llama2 has a larger transaction capacity than Monero, with a maximum of 50,000 transactions per block compared to Monero's 144 transactions per block. This could make Llama2 more suitable for high-volume transactions.
4. Privacy Feat

## Load multiple and process pdfs

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader('./input', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [7]:
len(documents)

40

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings

In [9]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-large-en-v1.5",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

In [10]:
from langchain.vectorstores import Chroma

persist_directory = 'db'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)

## Make a retriever

In [11]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Make a chain

In [12]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# create the chain to answer questions
system_template = """You are an LLM specialist. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""

        # Create the chat prompt templates
PROMPT = PromptTemplate(template=system_template, input_variables=['context', 'question'])

qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  chain_type_kwargs={"prompt": PROMPT},
                                  return_source_documents=True)

In [13]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [15]:
# full example
query = "What do you know about Llama2? Answer short in one sentence."
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Llama2 is a new open-source large language model developed by Meta AI that is still under development and
designed to be a versatile and powerful LLM for various tasks, including query resolution, natural language
comprehension, and reading comprehension.


Sources:
input/llms.pdf
input/embeddings.pdf
input/embeddings.pdf


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7d4060235d20>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)